In [14]:
import pandas as pd
import joblib as jb
from sklearn.preprocessing import LabelEncoder

### Import data and models

In [21]:
data = pd.read_csv(r'D:\0_Respaldo\0_Proyectos_2024\ML_proyects\KagelX\KaggelX_Challenge\data\test.csv')

In [17]:
label_engine = jb.load(r'D:\0_Respaldo\0_Proyectos_2024\ML_proyects\KagelX\KaggelX_Challenge\data\transform_data\label_engine.pkl')

In [30]:
label_accident = jb.load(r'D:\0_Respaldo\0_Proyectos_2024\ML_proyects\KagelX\KaggelX_Challenge\data\transform_data\label_accident.pkl')

In [44]:
# Load ML model
model = jb.load(r'D:\0_Respaldo\0_Proyectos_2024\ML_proyects\KagelX\KaggelX_Challenge\data\transform_data\ml.pkl')

In [22]:
df =data.copy()

In [23]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36183 entries, 0 to 36182
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            36183 non-null  int64 
 1   brand         36183 non-null  object
 2   model         36183 non-null  object
 3   model_year    36183 non-null  int64 
 4   milage        36183 non-null  int64 
 5   fuel_type     36183 non-null  object
 6   engine        36183 non-null  object
 7   transmission  36183 non-null  object
 8   ext_col       36183 non-null  object
 9   int_col       36183 non-null  object
 10  accident      36183 non-null  object
 11  clean_title   36183 non-null  object
dtypes: int64(3), object(9)
memory usage: 3.3+ MB


,id,model_year,milage
count,36183.000000,36183.000000,36183.000000
mean,72364.000000,2015.063953,72479.266755
std,10445.276732,5.589336,50714.968252
min,54273.000000,1974.000000,100.000000
25%,63318.500000,2012.000000,31681.000000
50%,72364.000000,2016.000000,65680.000000
75%,81409.500000,2019.000000,102000.000000
max,90455.000000,2024.000000,405000.000000


In [24]:
df.head(5)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,54273,Mercedes-Benz,E-Class E 350,2014,73000,Gasoline,302.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,White,Beige,None reported,Yes
1,54274,Lexus,RX 350 Base,2015,128032,Gasoline,275.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Silver,Black,None reported,Yes
2,54275,Mercedes-Benz,C-Class C 300,2015,51983,Gasoline,241.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Blue,White,None reported,Yes
3,54276,Land,Rover Range Rover 5.0L Supercharged Autobiogra...,2018,29500,Gasoline,518.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,White,At least 1 accident or damage reported,Yes
4,54277,BMW,X6 xDrive40i,2020,90000,Gasoline,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes


### Transformate df

In [ ]:
df.head(2)

In [31]:
df1 = df[['id','model_year', 'milage', 'engine', 'accident']].copy()

In [32]:
df1.head(2)

,id,model_year,milage,engine,accident
0,54273,2014,73000,302.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,None reported
1,54274,2015,128032,275.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,None reported


In [35]:
#Funcion para manejar las transformaciones.
def safe_transform(encoder, values):
    transformed_values = []
    for item in values:
        try:
            transformed_values.append(encoder.transform([item])[0])
        except ValueError:
            # Asigna un valor específico para valores no vistos, por ejemplo, -1
            transformed_values.append(-1)
    return transformed_values

In [37]:
df1['engine'] = safe_transform(label_engine, df['engine'])


In [39]:
df1['accident'] = safe_transform(label_accident, df['accident'])

In [40]:
df1.head(3)

,id,model_year,milage,engine,accident
0,54273,2014,73000,554,1
1,54274,2015,128032,414,1
2,54275,2015,51983,313,1


In [41]:
#columns defined
features = ['model_year', 'milage', 'engine','accident']
X = df1[features]

In [45]:
predictions = model.predict(X)

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [46]:
#Final dataframe
df_f = pd.DataFrame({
    'id':df['id'],
    'price': predictions
})

In [48]:
df_f['price'] = round(df_f['price'],2)

In [49]:
df_f.head(5)

,id,price
0,54273,24706.87
1,54274,19982.97
2,54275,26277.40
3,54276,51721.81
4,54277,34598.91


In [50]:
df_f.to_csv('Submission_File.csv', index=False)